# First script inversion of the ZNS parameters

Thanks to this script we will carry out an inversion of the parameters of the unsaturated area, the porosity and the depth of the water table in steady state from a reference simulation sampled in 2000 points. The heat transfer takes place in a one-dimensional soil of height 40m represented by 2000 cells. The load is imposed.

The inversion will focus on the four uncertain parameters and one boundary condition of this model, namely:

- the porosity $n$
- the alpha Van Genuchten parameter $\alpha$,
- the $n$ Van Genuchten parameter, 
- the residual saturation $s_wres$, and
- the water table altitude (WT).




# O. Initiation Ginbette files

In [357]:
#!/usr/bin/env python

import os
import numpy as np
from pathlib import Path
import pandas as pd
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import display
import subprocess
libs_gfortran = ['gfortran']
# please compile ginette in the folder 1D_col
# path of the 1D_col directory
os.chdir('/home/ariviere/Programmes/ginette/application/ZNS-1D/')
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

#-----------------------------------------------------------------
# Compile ginette
if os.path.isfile('ginette'):
    print ("ginette exist")
else:
    print ("ginette not exist")
    print("you must compile ginette in the current directory")
    print(" gfortran -o ginette ../../src/ginette_V2.f")
    subprocess.call(["gfortran","-o","ginette","../../src/ginette_V2.f"])  #creat

#-----------------------------------------------------------------
########### Setup of the model

#time step in s
dt=900
#duration of the simulation in days
nb_day=100

# state
## 0 steady state
# 1 transient state (dynamic state)
state=0

# size columnin meter
z_top=40

#discretisation : size cell in meter
dz=0.02

#apply unsaturated flow and thermal 
#unsat =1 apply
#unsat=0 cancel unsaturated zone
unsat=1

# number of facies in the column. If nb_zone=1 homognous porous media
nb_zone=1

# number of cell
nb_cell=z_top/dz
#-----------------------------------------------------------------
## write the setup of the moddeled domain
f_param_bck=open("E_parametre_backup.dat", "r")
f_param_new = open("E_parametre.dat", 'w')
setup_model=f_param_bck.read()
setup_model=setup_model.replace('[dt]','%06.0fD+00' % dt)
setup_model=setup_model.replace('[state]','%1i' % state)
setup_model=setup_model.replace('[nb_day]','%06.0f' % nb_day)
setup_model=setup_model.replace('[z_top]', '%7.3e' % z_top)
setup_model=setup_model.replace('[az]','%7.3e' % z_top)
setup_model=setup_model.replace('[dz]','%6.2e' % dz)
setup_model=setup_model.replace('[nb_cell]','%05.0f' % nb_cell)
setup_model=setup_model.replace('[unsat]','%1i' % unsat)

f_param_new.write(setup_model)
f_param_bck.close()
f_param_new.close()

#-----------------------------------------------------------------
# constant parameters
## intrinsic permeability [m2]  k=K*mu/(rho*g)
## K hydraulic conductivity [m.s-1]
## mu viscosity [Pa.s]
## rho density [kg.m-3]
## g gravity  9.81 [m2.s-1]
val_k=3.33333333333333e-15
# solid grain density rho_s=val_r  [kg.m-3]
val_r=2578


Current working directory: /home/ariviere/Programmes/ginette/application/ZNS-1D
ginette exist




## 1. Reference simulation and sampling

We choose to use the average parameters given by Carsel and Parrish (1988) as reference values for all facies


In [358]:
import numpy as np
import pandas as pd
import array

# porosity
REF_n=0.38 # \Phi

# Van Genuchten parameters
REF_a=2.70000 #m-1 alpha_vg
REF_nVG= 1.23  # n_vg
REF_swres=0.26 # S_wr

# Boundary condition
REF_WT=15


## 2. Screening parmeters
### 2.1 Range definition


In [359]:
dvt_a = 0.3
dvt_nVG = 0.1
dvt_swres = 0.1
dvt_n = 0.1
dvt_WT = 5



range_a = [REF_a-dvt_a,REF_a+dvt_a]
range_nVG =  [REF_nVG-dvt_nVG,REF_nVG+dvt_nVG]
range_swres =  [REF_swres-dvt_swres,REF_swres+dvt_swres]
range_n =  [REF_n-dvt_n,REF_n+dvt_n]
range_WT =  [REF_WT-dvt_WT,REF_WT+dvt_WT]

### 2.2 Paramétrisation

Choice of step values for each of the parameters, the measurement error and the number of sample. The following values are proposed as a first intention, they can be modified if necessary.

In [360]:
dvt_a = 0.1
dvt_nVG = 0.1
dvt_swres = 0.1
dvt_n = 0.1
dvt_WT = 0.1

# uncertainties
sigma = [0.002,0.006,0.81]

#Ratio of n samples
n_sample=[2,10,100]


### 2.3 loop table param
Later, you will use a loop of i over param_all


In [361]:
df = pd.DataFrame()
param_all=["a","n","nVG","swres","WT"]
param=["a"]
for i in param:
    df[i] = np.arange(range_a[0],range_a[1]+eval('dvt_{}'.format(i)),eval('dvt_{}'.format(i)))
    for x in param_all :
        if x != i:
            df[x] = eval('REF_{}'.format(x))
            
for s in sigma:
    df[s] = np.nan
    
for n in n_sample:
    df[n] = np.nan

df

,a,n,nVG,swres,WT,0.002,0.006,0.81,2,10,100
0,2.4,0.38,1.23,0.26,15,NaN,NaN,NaN,NaN,NaN,NaN
1,2.5,0.38,1.23,0.26,15,NaN,NaN,NaN,NaN,NaN,NaN
2,2.6,0.38,1.23,0.26,15,NaN,NaN,NaN,NaN,NaN,NaN
3,2.7,0.38,1.23,0.26,15,NaN,NaN,NaN,NaN,NaN,NaN
4,2.8,0.38,1.23,0.26,15,NaN,NaN,NaN,NaN,NaN,NaN
5,2.9,0.38,1.23,0.26,15,NaN,NaN,NaN,NaN,NaN,NaN
6,3.0,0.38,1.23,0.26,15,NaN,NaN,NaN,NaN,NaN,NaN


### 2.4 Direct model 

Read the REF simulation. Later it will be nice to optimize the script to loop over each facies. 

In [362]:
saturation_profile_REF = pd.read_table('/home/ariviere/Programmes/ginette/application/ZNS-1D/REF/S_saturation_profil_t.dat',delim_whitespace=True,header=None)
saturation_profile_REF.columns=[ "time",  "z","sat"]
saturation_profile_REF=saturation_profile_REF.head(2000)

### 2.5 Misfit function

In [363]:
def rmse(predictions, ref,sigma):
    differences = ((predictions - ref)/sigma)                      #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^

    return rmse_val

### 2.6 one simulation

One you're sure all steps work correctly, you will uncoment the loop over the whole table of parameters. Replace all row_1 by df

In [364]:
row_1=df.head(1)
#for index, row in df.iterrows():
for index, row in row_1.iterrows():
# Initial conditions
    f_IC_bck=open("E_cdt_initiale_backup.dat","r")
    IC_model=f_IC_bck.read()
    IC_model=IC_model.replace('[head_ini]', '%05.0fD+00' % row['WT'])
## write the boundary conditions
    f_bc_bck=open("E_cdt_aux_limites_backup.dat", "r")
    bc_model=f_bc_bck.read()
    bc_model=bc_model.replace('[top]', '%08.0fD+00' % row['WT'])
    bc_model=bc_model.replace('[bot]','%08.0fD+00' % row['WT'])
    ########### Zone of parameters
    f_coor=open("E_coordonnee.dat", "w")
    f_zone=open("E_zone.dat", 'w')
    f_paramZ_bck=open("E_zone_parameter_backup.dat", "r")
    f_paramZ_new = open("E_zone_parameter.dat", 'w')
    f_bc_new = open("E_cdt_aux_limites.dat", 'w')
    f_IC_new=open("E_cdt_initiale.dat","w")
    param_zone=f_paramZ_bck.read()
    coord=pd.DataFrame()    
#coord = pd.read_csv(f_coor, names=["id", "x", "z"], header=None, delim_whitespace=True)

# replace the parameter values
    param_zone=param_zone.replace('[k1]','%8.2e' % val_k)
    param_zone=param_zone.replace('[n1]','%6.2f' % row['n'])
    param_zone=param_zone.replace('[r1]','%6.2f' % val_r)
    param_zone=param_zone.replace('[a1]','%8.2e' % row['a'])
    param_zone=param_zone.replace('[nVG1]','%6.2f' % row['nVG'])
    param_zone=param_zone.replace('[swres1]','%6.2f' % row['swres'])
    
# calculate the coordinates to know the number of cell in the domain    
    zvalues = np.arange(dz/2, z_top,dz );
    xvalues = np.array([0.5]);
    zz, xx = np.meshgrid(zvalues, xvalues)
    NT = np.product(zz.shape)
    data = {
        "x": np.reshape(xx,NT),
        "z": np.reshape(zz,NT)}
    coord = pd.DataFrame(data=data)
    coord['id']=coord.index.values.astype(int)
    coord['id']=coord['id']+1
    cols = coord.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    coord = coord[cols] 
    coord.to_csv(f_coor, index = False, sep=' ', header=False)
#zone parameter by cell ((homogenous domain = 1 zone))
    coord['zone'] =1



#Write new ginette files
    f_IC_new.write(IC_model)
    f_paramZ_new.write(param_zone)
    f_bc_new.write(bc_model)
    coord.zone.to_csv(f_zone, index = False, header=False)
    
# close files    
    f_zone.close()
    f_coor.close()
    f_paramZ_new.close()
    f_paramZ_bck.close()

    f_IC_new.close()
    f_bc_bck.close()
    f_bc_new.close()
    f_coor.close()


# run ginette
#    subprocess.call(["./ginette"])
    
# read the saturation profil   
    saturation_profile = pd.read_table('S_saturation_profil_t.dat',delim_whitespace=True,header=None)
    saturation_profile.columns=[ "time",  "z","sat"]

# Calculate the MISFITS
## function of uncertainties        
    for s in sigma:
        row_1.at[index , s] = rmse(saturation_profile['sat'],saturation_profile_REF['sat'],s)

## function of size of sample
    for n in n_sample:
        sim=saturation_profile.loc[::n,'sat']
        ref=saturation_profile_REF.loc[::n,'sat']
        row_1.at[index , n_sample] = rmse(sim,ref,sigma[0])
        



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [366]:
row_1

,a,n,nVG,swres,WT,0.002,0.006,0.81,2,10,100
0,2.4,0.38,1.23,0.26,15,0.0,0.0,0.0,0.0,0.0,0.0
